In [2]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
# Input : list of stocks to short and long at time t
long = ['AAPL']

short = ['MMM']

In [14]:
# Import earning calls
tickers = long+short

texts = []
list_ids = []
Ticker = []
number_texts_per_stock = []
for ticker in tickers:
    dir_path = "data/text/"+ticker+"/"
    list_ids1 = os.listdir(dir_path)
    ticker1 = os.path.basename(os.path.dirname(dir_path))
    ticker1 = ticker1.split()
    texts1 = []
    for s in list_ids1:
        with open(dir_path + s, encoding = 'ISO-8859-1') as f:
            t = f.read()
            texts1.append(t)
    texts = texts + texts1
    list_ids = list_ids + list_ids1 
    Ticker = Ticker + ticker1*len(list_ids1)
    number_texts_per_stock.append(len(list_ids1))

# Get dates
data_path = "data/ticker/id_to_date"
with open(data_path,"rb") as file :
    id_to_date = pickle.load(file)
    
# Create dataframe with earning calls informations
df1 = pd.DataFrame(data={'id': list_ids})
df1['date']=list_ids
df1=df1.replace({'date': id_to_date})
df1['date'] = pd.to_datetime(df1['date'])
df1['quarter'] = df1['date'].dt.to_period('Q')
df1['ticker']= Ticker
df1=df1.replace({'ticker': id_to_date})
df1= df1.sort_values(['ticker','date'], ascending=True)

# Import datas from yahoo finance

In [15]:
# Get the month from which we import
# Q1 : from january (1) to march (3)
# Q2 : from april (4) to june (6)
# Q3 : from july (7) to septmber (9)
# Q4 : from october (10) to december (12)

quarter_start = df1['quarter'].iloc[0].quarter
month_start = quarter_start*2-1
quarter_end = df1['quarter'].iloc[-2].quarter # take the last quarter
month_end = quarter_end*3

if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30

In [48]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

start = datetime.datetime(df1['date'].iloc[0].year,month_start,1)
end   = datetime.datetime(df1['date'].iloc[-1].year,month_end,day_end)

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
list_tickers = long+short
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
stock_final['date']=stock_final.index
stock_final['quarter'] = stock_final['date'].dt.to_period('Q')
stock_final = stock_final.drop(columns = ['date','Open','High','Low','Close','Volume'])
display(stock_final)

0 : AAPL,1 : MMM,

,Adj Close,ticker,quarter
Date,,,
2006-01-03,2.282592,AAPL,2006Q1
2006-01-04,2.289310,AAPL,2006Q1
2006-01-05,2.271295,AAPL,2006Q1
2006-01-06,2.329923,AAPL,2006Q1
2006-01-09,2.322290,AAPL,2006Q1
...,...,...,...
2022-03-24,147.489502,MMM,2022Q1
2022-03-25,148.954697,MMM,2022Q1
2022-03-28,147.974594,MMM,2022Q1


In [50]:
stock_final = stock_final.groupby(['ticker','quarter']).mean()
stock_final['return'] = stock_final['Adj Close'].pct_change()
stock_final

Adj Close    return
ticker quarter                      
AAPL   2006Q1     2.142406       NaN
       2006Q2     1.955243 -0.087361
       2006Q3     2.021831  0.034056
       2006Q4     2.529399  0.251043
       2007Q1     2.703083  0.068666
...                    ...       ...
MMM    2021Q1   170.942407  0.064402
       2021Q2   191.891548  0.122551
       2021Q3   187.665464 -0.022023
       2021Q4   173.947825 -0.073096
       2022Q1   156.759947 -0.098811

[130 rows x 2 columns]

In [ ]:
stock_final